# Description

This architecture was described in "Deep learning with convolutional neural networks for brain mapping and decoding of movement-related information from the human EEG", by R. T. Schirrmeister et al, 2018. In this notebook we conduct experiments showing dependency between accuracy and the number of timestamps in a sample. 

# Set up the environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [3]:
cd ..

/Users/sriramsonti/Desktop/project_C247


# Load the data

### Read the files

In [4]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [5]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [6]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Preprocess data

In [7]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

# Experiment 1: shallow model 

In this experiment we show that:
1. shallow model can achieve up to 61.5% of validation accuracy given samples with 1000 timestamps;
2. shallow model achieves lowest validation loss after being trained for 9 epochs, after which it starts to overfit.

### define activation functions

In [8]:
def Ksquare(x):
    return K.pow(x, 2)

def Klog(x):
    return K.log(x)

### Construct model

In [9]:
def construct_shallow_model(TIME_WINDOW):
    # input
    shallow_input = layers.Input(shape=(22, TIME_WINDOW))

    # conv accross time domain
    r1 = layers.Reshape((22, TIME_WINDOW, 1))(shallow_input)
    c1 = layers.Conv2D(40, (1, 25), strides=(1, 1), activation="elu")(r1)
    new_size = TIME_WINDOW - 25 + 1
    t1 = tf.keras.layers.Permute((2, 3, 1))(c1)
    
    
    # conv accross time domain
    r2 = layers.Reshape((new_size, 40*22, 1))(t1)
    c2 = layers.Conv2D(40, (1, 40*22), strides=(1, 1), activation="elu")(r2)

    sq1 = layers.Activation(Ksquare)(c2)
    r3 = layers.Reshape((new_size, 40, 1))(sq1)
    apool1 = layers.AveragePooling2D(pool_size=(75, 1), strides=(15, 1))(r3)

    log1 = layers.Activation(Klog)(apool1)
    f1 = layers.Flatten()(log1)

    # output
    shallow_output = layers.Dense(4, activation="softmax")(f1)
    
    return keras.Model(inputs = shallow_input, outputs = shallow_output)

In [10]:
shallow_model_1000 = construct_shallow_model(1000)
shallow_model_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

In [11]:
shallow_model_1000.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
reshape (Reshape)            (None, 22, 1000, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 22, 976, 40)       1040      
_________________________________________________________________
permute (Permute)            (None, 976, 40, 22)       0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 976, 880, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 976, 1, 40)        35240     
_________________________________________________________________
activation (Activation)      (None, 976, 1, 40)        0     

### Make checkpoints

In [12]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

### Train model on single person

In [14]:
person_num = 0
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

Training data shape for 1 person: (189, 22, 1000)
Training label shape for 1 person: (189,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)


In [16]:
shallow_model_1000_single = construct_shallow_model(1000)
shallow_model_1000_single.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])
shallow_model_1000_single.fit(single_person_X_train, single_person_y_train,
                                            validation_data = (single_person_X_valid, single_person_y_valid),
                                            epochs = 30)

Train on 189 samples, validate on 48 samples
Epoch 1/30
189/189 [==============================] - 15s 78ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 2/30
189/189 [==============================] - 19s 99ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 3/30
189/189 [==============================] - 17s 90ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 4/30
189/189 [==============================] - 15s 79ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 5/30
189/189 [==============================] - 14s 73ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 6/30
189/189 [==============================] - 13s 71ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 7/30
189/189 [==============================] - 14s 73ms/sample - loss: nan - acc: 0.2646 - val_loss: nan - val_acc: 0.2083
Epoch 8/30
189/189 [=========================

In [17]:
shallow_model_1000_single.evaluate(single_person_X_test, single_person_y_test)

50/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[nan, 0.24]

In [18]:
shallow_model_1000_single.evaluate(X_test, y_test)

443/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[nan, 0.25056434]

### Train model

In [21]:
shallow_model_loss_hist = shallow_model_1000.fit(X_train, y_train,
                                                 validation_data = (X_valid, y_valid),
                                                 epochs = 30,
                                                 callbacks=checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/30
1664/1692 [============================>.] - ETA: 1s - loss: nan - acc: 0.2542
Epoch 00001: val_loss did not improve from inf
1692/1692 [==============================] - 116s 68ms/sample - loss: nan - acc: 0.2565 - val_loss: nan - val_acc: 0.2246
Epoch 2/30
1664/1692 [============================>.] - ETA: 1s - loss: nan - acc: 0.2560
Epoch 00002: val_loss did not improve from inf
1692/1692 [==============================] - 122s 72ms/sample - loss: nan - acc: 0.2565 - val_loss: nan - val_acc: 0.2246
Epoch 3/30
 544/1692 [========>.....................] - ETA: 1:18 - loss: nan - acc: 0.2637WARNING:tensorflow:Can save best model only with val_loss available, skipping.


KeyboardInterrupt: 

### Produce graphs

In [ ]:
hist = shallow_model_loss_hist.history

plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])

plt.subplot(1, 2, 2)
plt.plot(hist['acc'])
plt.plot(hist['val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'])

# Experiment 2: shallow model with normilized data.

In this experiment we shaow that data normalization does not help to improve validation accuracy. For the following experiment with shallow model, we do not try to normilize data.

### Prepare data

In [ ]:
# normilize the data USING ONLY TRAIN DATA MEAN AND STANDARD DEVIATION
X_train_norm = (X_train - np.mean(X_train))/np.std(X_train)
X_valid_norm = (X_valid - np.mean(X_train))/np.std(X_train)
X_test_norm = (X_test - np.mean(X_train))/np.std(X_train)

### Construct model

In [ ]:
shallow_model_1000_norm = construct_shallow_model(1000)
shallow_model_1000_norm.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

### Make checkpoints

In [ ]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_1000_norm',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

### Train model

In [ ]:
shallow_model_1000_norm.fit(X_train_norm, y_train,
                            validation_data = (X_valid_norm, y_valid),
                            epochs = 30,
                            callbacks=checkpoint_callback)

# Experiment 3: shallow model - accuracy vs number of timestamps 

In this experiment we show that accuracy peaks at 700 timestamps per sample

### Train models

In [ ]:
TIME_WINDOW = 300
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_300 = construct_shallow_model(TIME_WINDOW)
shallow_model_300.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_300.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
TIME_WINDOW = 500
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_500 = construct_shallow_model(TIME_WINDOW)
shallow_model_500.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_500.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_600 = construct_shallow_model(TIME_WINDOW)
shallow_model_600.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_600.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
TIME_WINDOW = 700
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_700 = construct_shallow_model(TIME_WINDOW)
shallow_model_700.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_700.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
TIME_WINDOW = 800
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_800 = construct_shallow_model(TIME_WINDOW)
shallow_model_800.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_800.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
TIME_WINDOW = 900
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train, 
                                                y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/shallow_model_' + str(TIME_WINDOW),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


shallow_model_900 = construct_shallow_model(TIME_WINDOW)
shallow_model_900.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_900.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30,
                      callbacks=checkpoint_callback)

In [ ]:
best_shallow_model_300 = keras.models.load_model('./model_checkpoints/shallow_model_300')
best_shallow_model_500 = keras.models.load_model('./model_checkpoints/shallow_model_500')
best_shallow_model_600 = keras.models.load_model('./model_checkpoints/shallow_model_600')
best_shallow_model_700 = keras.models.load_model('./model_checkpoints/shallow_model_700')
best_shallow_model_800 = keras.models.load_model('./model_checkpoints/shallow_model_800')
best_shallow_model_900 = keras.models.load_model('./model_checkpoints/shallow_model_900')
best_shallow_model_1000 = keras.models.load_model('./model_checkpoints/shallow_model_1000')

number_of_samples = [300, 500, 600, 700, 800, 900, 1000]
accuracies = []


# ==================================== 300 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=300, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_300.evaluate(X_valid_slices, y_valid_slices)[1])


# ==================================== 500 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=500, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_500.evaluate(X_valid_slices, y_valid_slices)[1])



# ==================================== 600 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=600, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_600.evaluate(X_valid_slices, y_valid_slices)[1])



# ==================================== 700 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=700, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_700.evaluate(X_valid_slices, y_valid_slices)[1])


# ==================================== 800 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=800, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_800.evaluate(X_valid_slices, y_valid_slices)[1])


# ==================================== 900 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=900, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_900.evaluate(X_valid_slices, y_valid_slices)[1])


# ==================================== 1000 ==================================== #

X_valid_slices, y_valid_slices = sliding_window(X_valid, 
                                                y_valid, 
                                                time_window=1000, 
                                                time_stride=TIME_STRIDE)

accuracies.append(best_shallow_model_1000.evaluate(X_valid_slices, y_valid_slices)[1])


# Plot the results

In [ ]:
fig = plt.figure()
ax = plt.axes()

plt.xlabel('number of samples')
plt.ylabel('accuracy')
ax.plot(number_of_samples, accuracies);


# Results

In [ ]:
X_test_slices, y_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=700, 
                                              time_stride=TIME_STRIDE)

shallow_model_results = best_shallow_model_700.evaluate(X_test_slices, y_test_slices)


print('\n# Evaluate on test data')
print('Optimal shallow model test loss:', shallow_model_results[0])
print('Optimal shallow model test acc:', shallow_model_results[1])